In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import utils
import skutils
from sklearn.preprocessing import StandardScaler

# Use ggplot style
plt.style.use('ggplot')

%matplotlib inline

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
big = train.append(test)
big.shape

(1584, 194)

In [5]:
def transform(data):
    ID = data.id
    X = data.drop(['species', 'id'], axis=1)
    y = data['species']
    return ID, X, y

ID, X, y = transform(big)

In [13]:
def addZeroColumn(df, colName):
    df.loc[df[colName] < 0.01, colName + '_is_small'] = 1
    df[colName + '_is_small'].fillna(0, inplace=True)

def addZeroColumns(df, colBaseName):
    for n in range(1,65):
        addZeroColumn(df, colBaseName + str(n))
        
# addZeroColumns(X, 'margin')
# addZeroColumns(X, 'texture')

colName = 'margin1'
X.loc[X[colName] < 0.01, colName + '_is_small'] = 1

AttributeError: 'numpy.ndarray' object has no attribute 'loc'

In [7]:
X = StandardScaler().fit(X).transform(X)

In [ ]:
big.columns

In [ ]:
X[]